Importaciones del Modelo

In [501]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from joblib import dump
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import r2_score
from keras.optimizers import Adagrad 
from keras.optimizers import Adadelta 
from keras.optimizers import RMSprop
from keras.optimizers import SGD 
from sklearn.metrics import mean_absolute_error

import numpy as np

import os

Creación de Carpeta de Resultados e importación de los Datos con Panda

In [502]:
file_path = 'MixOMP(MissingValues).csv'
df = pd.read_csv(file_path)#Guardar los datos en dataframe->df
RESULT_FOLDER='../Resultados_Modelo/'

if not os.path.exists(RESULT_FOLDER):
    os.mkdir(RESULT_FOLDER)
columnas_deseadasForest = ['Time','N','Recur','Max_Consumo_Memoria','Promedio_PorCiento_Uso_Proc','Promedio_Frecuencia_Proc']
#df.columns = range(df.shape[1])
df=df.dropna()
#datos_seleccionadosForest = df['Promedio_PorCiento_Uso_Proc']
print(df.columns)

Index(['Time', 'N', 'Recur', 'Hilos', 'Inicio', 'Fin', 'Max_Consumo_Memoria',
       'PorCiento_Uso_Memoria', 'Promedio_PorCiento_Uso_Proc',
       'Max_PorCiento_Uso_Proc', 'Promedio_Temp', 'Max_Temp',
       'Promedio_Frecuencia_Proc', 'Max_Frecuencia'],
      dtype='object')


Importación del Modelo RandomForest

In [503]:
model_regression=RandomForestRegressor(random_state=42)
#Escoger los valores según la estrategia de calendarización (Secuencial,OMP_Recursivo,OMP_NoRecursivo,CUDA,CUDA_OMP_Recursivo)

#Empezar por el secuencial
valor_especifico = 'CUDA'

# Filtrar los datos
#datos_filtrados = df[df['MODEL'] == valor_especifico]
columnas_deseadasForest = ['Time','N','Recur','Max_Consumo_Memoria','Promedio_PorCiento_Uso_Proc','Promedio_Frecuencia_Proc']
columnas_deseadas = ['Time','N','Recur','Hilos','Max_Consumo_Memoria','PorCiento_Uso_Memoria','Promedio_PorCiento_Uso_Proc','Max_PorCiento_Uso_Proc','Promedio_Frecuencia_Proc','Max_Frecuencia']
# Seleccionar las columnas deseadas
datos_seleccionados = df.loc[:, columnas_deseadas]
datos_seleccionadosForest = df.loc[:, columnas_deseadasForest]

# Crear una instancia del escalador StandardScaler
scalers = {}
# Iterar sobre cada columna y aplicar el scaler correspondiente
for col in columnas_deseadas:
    # Crear un scaler para la columna actual
    scaler = StandardScaler()
    # Obtener los valores de la columna y reshape para que sea un array 2D
    columna = datos_seleccionados[col].values.reshape(-1, 1)
    # Normalizar la columna actual y guardar el scaler en el diccionario
    columna_normalizada = scaler.fit_transform(columna)
    scalers[col] = scaler
    # Agregar la columna normalizada al DataFrame df_normalizado
    datos_seleccionados[col] = columna_normalizada.flatten()
   
#Normalizar utilizando z_score
#df_normalizado = pd.DataFrame(scaler.fit_transform(datos_seleccionados), columns=datos_seleccionados.columns)
#Dividir los datos normalizados en prueba y entrenamiento
X= datos_seleccionados.loc[:, columnas_deseadas[1:len(columnas_deseadas)]]
Y=datos_seleccionados.loc[:,columnas_deseadas[0]]
print(len(columnas_deseadas))
Xf= datos_seleccionadosForest.loc[:, columnas_deseadasForest[1:len(columnas_deseadasForest)]]
Yf=datos_seleccionadosForest.loc[:,columnas_deseadasForest[0]]


X_trainF, X_testF, y_trainF, y_testF = train_test_split(Xf, Yf, test_size=0.2, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X.values, Y.values, test_size=0.2, random_state=42)
#Probar con LSTM
# Dividir los datos en conjuntos de entrenamiento y prueba
# Definir el número de pasos de tiempo para la secuencia de entrada
time_steps = 10
# Preparar los datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_trainLstm = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_testLstm = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
y_trainLstm = np.reshape(y_train, (-1, 1))
y_testLstm = np.reshape(y_test, (-1, 1))
#print(X_trainLstm)
optimizer=Adagrad(learning_rate=0.01)#regular mse=0.09 r2=0.70
optimizer2=Adadelta(learning_rate=0.01)#malo
optimizer3=RMSprop(learning_rate=0.01)#buenisimo mse=0.01 r2=0.96
optimizer4=SGD(learning_rate=0.01,momentum=0.99)#bueno mse=0.04 r2=0.85
#print(X_trainLstm)
#inpua=input_shape=(1, X.shape[1])
#print(X_trainLstm[0][0])
print(X_train.head())

10
            N     Recur     Hilos  Max_Consumo_Memoria  PorCiento_Uso_Memoria  \
237 -0.881070 -0.044663  1.022434            -0.834588              -0.828864   
703  0.918267  0.917144  0.443013             1.303282               1.301555   
598  0.404171 -1.006469  1.312145             0.014341               0.014427   
896  1.689411 -1.006469  0.732724            -0.013393              -0.029957   
474 -0.109926  0.917144  0.153303             0.014165               0.014427   

     Promedio_PorCiento_Uso_Proc  Max_PorCiento_Uso_Proc  \
237                     1.181294                1.028477   
703                     0.308846                0.331797   
598                     1.454949                1.358089   
896                     0.613309                0.743812   
474                     0.314375                0.103315   

     Promedio_Frecuencia_Proc  Max_Frecuencia  
237                 -0.005995       -0.653964  
703                 -0.197148        0.641880  
598  

In [504]:
#SecuencialPuro
modelSecuencial = Sequential()
modelSecuencial.add(Dense(9,input_dim=9,kernel_initializer='normal'))
"""modelSecuencial.add(Dense(100,kernel_initializer='normal'))
modelSecuencial.add(Dense(60,kernel_initializer='normal'))
modelSecuencial.add(Dense(100,kernel_initializer='normal'))
"""
modelSecuencial.add(Dense(60,kernel_initializer='normal'))
modelSecuencial.add(Dense(1,kernel_initializer='normal'))
modelSecuencial.compile(loss='mean_squared_error', optimizer=optimizer3)
modelSecuencial.fit(X_train, y_train, epochs=60, batch_size=32)

# Evaluar el modelo con los datos de prueba
result = modelSecuencial.evaluate(X_test, y_test)  # Ajusta los datos de prueba adecuados
print("Modelo Secuencial Puro"+str(result))
# Crear un modelo secuencial para LSTM
model = Sequential()
# Agregar una capa LSTM
model.add(LSTM(units=80, input_shape=(1, X.shape[1])))
# Agregar una capa de salida
model.add(Dense(units=1))
# Compilar el modelo
model.compile(loss='mean_squared_error', optimizer=optimizer3)

# Entrenar el modelo
model.fit(X_trainLstm, y_train, epochs=60, batch_size=32)

# Evaluar el modelo en los datos de prueba
mse = model.evaluate(X_testLstm, y_test)
print("Error cuadrático medio (MSE):", mse)
#Probar con Random Forest
model_regression.fit(X_trainF, y_trainF)
score=model_regression.score(X_testF,y_testF)
predictionsRandomForest=model_regression.predict(X_testF)
mae = mean_absolute_error(y_testF, predictionsRandomForest)
mape = np.mean(np.abs((y_testF - predictionsRandomForest) / y_testF)) * 100

print("Random Forest: " + str(score))

print("Random Forest: " + str(mae))

print("Random Forest: " + str(mape))

predictionsLstm = model.predict(X_testLstm)
#y_testLstm=np.reshape(-1, 1)
#print(y_test)
desnormalized_predictionsLstm=scalers['Time'].inverse_transform(predictionsLstm)
reshape_forest_prediction = np.reshape(predictionsRandomForest, (-1, 1))
desnormalized_real_values=scalers['Time'].inverse_transform(y_testLstm)
desnormalized_predictionsRandomForest=scalers['Time'].inverse_transform(reshape_forest_prediction)
print("LSTM Predicciones")
print(desnormalized_predictionsLstm)
print("Valores Reales")
print(y_testF)
print("Random Forest Predicciones")
print(predictionsRandomForest)
r2 = r2_score(y_testLstm, predictionsLstm)
print("Coeficiente de determinación (R cuadrado) para LSTM:", r2)


Epoch 1/60
24/24 [==============================] - 0s 2ms/step - loss: 0.7484
Epoch 2/60
24/24 [==============================] - 0s 2ms/step - loss: 0.5444
Epoch 3/60
24/24 [==============================] - 0s 3ms/step - loss: 0.4909
Epoch 4/60
24/24 [==============================] - 0s 4ms/step - loss: 0.4709
Epoch 5/60
24/24 [==============================] - 0s 4ms/step - loss: 0.4757
Epoch 6/60
24/24 [==============================] - 0s 2ms/step - loss: 0.4622
Epoch 7/60
24/24 [==============================] - 0s 1ms/step - loss: 0.4683
Epoch 8/60
24/24 [==============================] - 0s 5ms/step - loss: 0.4695
Epoch 9/60
24/24 [==============================] - 0s 4ms/step - loss: 0.4728
Epoch 10/60
24/24 [==============================] - 0s 797us/step - loss: 0.4782
Epoch 11/60
24/24 [==============================] - 0s 3ms/step - loss: 0.4749
Epoch 12/60
24/24 [==============================] - 0s 3ms/step - loss: 0.4719
Epoch 13/60
24/24 [============================

2023-07-16 05:05:24.413793: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-16 05:05:24.414784: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-16 05:05:24.415460: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

24/24 [==============================] - 1s 2ms/step - loss: 0.5179
Epoch 2/60
24/24 [==============================] - 0s 2ms/step - loss: 0.3246
Epoch 3/60
24/24 [==============================] - 0s 2ms/step - loss: 0.3231
Epoch 4/60
24/24 [==============================] - 0s 2ms/step - loss: 0.2423
Epoch 5/60
24/24 [==============================] - 0s 2ms/step - loss: 0.2360
Epoch 6/60
24/24 [==============================] - 0s 3ms/step - loss: 0.2272
Epoch 7/60
24/24 [==============================] - 0s 2ms/step - loss: 0.2386
Epoch 8/60
24/24 [==============================] - 0s 2ms/step - loss: 0.1996
Epoch 9/60
24/24 [==============================] - 0s 2ms/step - loss: 0.2102
Epoch 10/60
24/24 [==============================] - 0s 2ms/step - loss: 0.1672
Epoch 11/60
24/24 [==============================] - 0s 2ms/step - loss: 0.1996
Epoch 12/60
24/24 [==============================] - 0s 2ms/step - loss: 0.1816
Epoch 13/60
24/24 [==============================] - 0s 2ms/

2023-07-16 05:05:27.964757: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-16 05:05:27.965724: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-16 05:05:27.966396: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Error cuadrático medio (MSE): 0.05642586201429367
Random Forest: 0.9491521255712336
Random Forest: 14.85578864759621
Random Forest: 18.763510073942445
6/6 [==============================] - 0s 2ms/step
LSTM Predicciones
[[-2.3961098e+00]
 [ 4.6324348e+01]
 [ 8.7765284e+00]
 [ 1.6327728e+02]
 [ 2.1765724e+01]
 [ 2.3320839e+01]
 [-3.4972680e+00]
 [-3.2328560e+00]
 [ 2.5193260e+01]
 [ 7.2345146e+01]
 [-4.8119421e+00]
 [ 1.6858371e+02]
 [ 1.9441528e+01]
 [ 8.7369699e+00]
 [ 6.5055857e+00]
 [-2.5487053e+00]
 [ 8.8365946e+00]
 [-6.5846169e-01]
 [ 1.1410008e+02]
 [ 4.6297474e+01]
 [ 3.2117439e+01]
 [-6.6585379e+00]
 [ 2.5316170e+02]
 [ 1.8095766e+02]
 [ 1.4639825e+00]
 [ 1.7496864e+01]
 [ 7.3140160e+01]
 [ 1.4710063e+02]
 [ 5.2864441e+02]
 [ 1.2768842e+03]
 [ 3.8870749e+02]
 [-4.2074728e+00]
 [ 5.7131859e+01]
 [ 1.8816563e+02]
 [ 3.0124474e+01]
 [ 2.8057549e+01]
 [-1.1413604e+01]
 [-3.5778191e+00]
 [ 1.6544356e+02]
 [-7.0339346e+00]
 [ 3.1316782e+02]
 [ 9.7528925e+00]
 [ 4.4339127e+01]
 [ 3.9

2023-07-16 05:05:28.389374: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-16 05:05:28.391545: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-16 05:05:28.392715: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus